In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

In [2]:
print('\n\n\tImportação de Fertilizantes\n\n')



	Importação de Fertilizantes




Fonte: Comexstat(http://comexstat.mdic.gov.br/pt/geral), portal para acesso gratuito às estatísticas de\n\
comércio exterior do Brasil, com os dados utilizados para construção da balança comercial brasileira, detalhados\n\
por NCM ou por Municípios do exportador/importador.\n\n')

O site informa que os dados estão disponíveis em arquivos compatados que contém arquivos CSV, utilizando como separador o ponto e vírgula (;), e com os nomes das colunas na primeira linha. 

Para esse projeto, o interesse esta sobre a base de dados denominada IMP_COMPLETA.zip, que contém histórico mensal das importações detalhadas por produto (código NCM), Unidade Federativa importadora, ano, mês, país de origem, quilograma líquido, valor dólar FOB (US$) e outras.

Para aproveitamento completo dos arquivos de dados, será necessário baixar e utilizar as Tabelas Auxiliares contendo a "Correlações de Códigos e Classificações". 

O arquivo principal e as tabelas auxiliares foram baixadas e salvas localmente na pasta '~/git/BD/Comexstat/'.

> **Nota em relação ao Github:** Os arquivos salvos **nesta fase** são maiores do que o tamanho aceito em minha conta, e por isso eles serão ignorados e não disponibilizados.

A classificação de produto mais abrangente disponível é a SH2 (Sistema Harmonizado) e no nome dela se buscará identificar palavras como "fertilizantes" ou "adubo".m

In [3]:
sh=pd.read_csv('~/git/BD/Comexstat/NCM_SH.csv',sep=';',encoding='Latin-1')

> Pode-se ver que a coluna CO_SH2 apresenta o código mais abrangente. Ainda pode-se ver que na coluna NO_SH2_POR existe o nome da categoria em português.
>
> Identifica-se a seguir qual SH2 tem a palavra "fertilizante" incluída no nome.

In [4]:
# Identifica qual nome de categoria apresenta a palavra "fertilizante"
sh2_fert=sh[sh.NO_SH2_POR.str.contains('fertilizante')]['CO_SH2'].unique()
sh2_fert[0]

31

In [5]:
# Identifica o código da única categoria 'fertilizante'
sh2_fert=sh2_fert[0]

In [6]:
print(f'\nNo site Comexstat, na lista de categorias mais abrangente (SH2), foi localizado um produto de nome "fertilizante"\n\
e de código igual a {sh2_fert}.\n\n\
Esse nome e código serão utilizados para filtrar a extração dos registros do arquivo compactado IMP_COMPLETA.zip,\n\
disponibilizado pelo site.\n\n')


No site Comexstat, na lista de categorias mais abrangente (SH2), foi localizado um produto de nome "fertilizante"
e de código igual a 31.

Esse nome e código serão utilizados para filtrar a extração dos registros do arquivo compactado IMP_COMPLETA.zip,
disponibilizado pelo site.




In [7]:
sh6_fert=sh[sh.CO_SH2==sh2_fert]['CO_SH6'].unique()

> Passaremos agora para a próxima tabela, a tabela NCM.csv, e dela extrairemos a lista de NCMs que se relacionam com a lista de SH6 dos fertilizantes. 

In [8]:
ncm=pd.read_csv('~/git/BD/Comexstat/NCM.csv', sep=';',encoding='latin1')
ncm_fert=ncm[ncm.CO_SH6.isin(sh6_fert)].CO_NCM.unique()

Agora será extraido do arquivo principal (IMP_COMPLETA.csv) os registros relativos aos NCMs listados.

As linhas filtradas serão concatenadas para formar o novo  **arquivo principal**, que em seguida será salvo no endereço '~/comex/data/imp1.csv'.

In [9]:
# Cria leitor do arquivo principal em pedaços
csv_reader=pd.read_csv('~/git/BD/Comexstat/IMP_COMPLETA.zip', sep=';', chunksize=1000000)

# Prepara tabela para receber e concatenar os filtrados dos pedaços
q=pd.DataFrame()

# Lê, filtra e concatena cada pedaço
for chunk in csv_reader:
    q=pd.concat([q, chunk[chunk.CO_NCM.isin(ncm_fert)]])

# Ordena os dados
imp1=q.sort_values(by=q.columns.to_list()).reset_index(drop=True)

directory = '~/git/Pricing-Fertilizer-take-Rains-into-account/temp/'

# Expand the user path (if '~' is used)
expanded_directory = os.path.expanduser(directory)

# Check if the directory exists
if not os.path.exists(expanded_directory):
    # Create the directory if it doesn't existyy
    os.makedirs(expanded_directory)

In [10]:
print('\n\nTabela de Importação de Fertilizantes\n')
display(imp1)
print()



Tabela de Importação de Fertilizantes



,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,1997,1,31010000,10,386,SP,1,817800,15222,15222,34324,1323,177
1,1997,1,31010000,10,573,SP,1,817800,1800,20000,7169,2638,0
2,1997,1,31021010,10,850,RR,7,260151,640,32000,7360,1120,0
3,1997,1,31021090,10,850,RR,7,260151,96000,96000,22080,3360,0
4,1997,1,31022100,10,23,GO,1,817800,1900000,1900000,184300,19000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100425,2024,1,31059090,10,676,SP,1,817800,72000,72000,170846,10766,111
100426,2024,1,31059090,10,827,ES,1,917800,60000,60000,55200,600,25
100427,2024,1,31059090,10,827,MG,1,917800,40000,40000,36800,400,17
100428,2024,1,31059090,10,827,SP,1,817800,282000,282000,356422,7120,140


A coluna SG_UF_NCM apresenta as siglas das Unidades Federativas do Brasil

In [11]:
print('\n\nEm nome da simplicidade, esse projeto abordará uma única unidade federativa do Brasil.\n\n\
A unidade federativa da Bahia foi a escolhida pelo autor para foco deste projeto.\n\n\
Serão somente salvas as importações destinadas a este estado brasileiro.\n\n')



Em nome da simplicidade, esse projeto abordará uma única unidade federativa do Brasil.

A unidade federativa da Bahia foi a escolhida pelo autor para foco deste projeto.

Serão somente salvas as importações destinadas a este estado brasileiro.




In [12]:
# Filtra deixando passar somente registros destinados a Bahia
imp1=imp1[imp1.SG_UF_NCM=='BA']

In [13]:
# Salva o resultado
imp1.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/imp1.csv', index=False)

# Chuvas

In [14]:
print('\n\n\tVolumes de Chuvas\n\n')



	Volumes de Chuvas




In [15]:
print('\n\nNo Estado da Bahia o INMET mantém várias estações meteorológicas automáticas,\n\
sendo essas identificadas pelo nome do município em que estão localizadas.\n\n\
\
Para selecionar as estações do INMET que mais se correlacionem com a importação de fertilizantes,\n\
é necessário identificar quais os municípios com maior área plantada:\n\n\
\
Rank dos Municípios com Maior Área Plantada no Estado da Bahia no ano 2022(Fonte: IBGE):\n\n\
\t*São Desidério (BA)\n\
\t*Formosa do Rio Preto (BA)\n\
\t*Correntina (BA)\n\
\t*Barreiras (BA)\n\
\t*Luís Eduardo Magalhães (BA)\n\
\t*Riachão das Neves (BA)\n\
\t*Jaborandi (BA)\n\n\
\
Comparando o rank acima com as estações meteorológicas listadas pelo INMET, identificamos as seguintes\n\
estações como de interesse do projeto:\n\n\
\
Estações meteorológicas de interesse:\n\n\
\t* FORMOSA DO RIO PRETO\n\
\t* CORRENTINA\n\
\t* LUIS EDUARDO MAGALHÃES\n\
\t* BARREIRAS')



No Estado da Bahia o INMET mantém várias estações meteorológicas automáticas,
sendo essas identificadas pelo nome do município em que estão localizadas.

Para selecionar as estações do INMET que mais se correlacionem com a importação de fertilizantes,
é necessário identificar quais os municípios com maior área plantada:

Rank dos Municípios com Maior Área Plantada no Estado da Bahia no ano 2022(Fonte: IBGE):

	*São Desidério (BA)
	*Formosa do Rio Preto (BA)
	*Correntina (BA)
	*Barreiras (BA)
	*Luís Eduardo Magalhães (BA)
	*Riachão das Neves (BA)
	*Jaborandi (BA)

Comparando o rank acima com as estações meteorológicas listadas pelo INMET, identificamos as seguintes
estações como de interesse do projeto:

Estações meteorológicas de interesse:

	* FORMOSA DO RIO PRETO
	* CORRENTINA
	* LUIS EDUARDO MAGALHÃES
	* BARREIRAS


O [INMET](https://portal.inmet.gov.br/) - Instituto Nacional de Meteorologia, subordinado ao Ministério da Agricultura e Pecuária, disponibiliza no site os dados históricos de chuvas em estações meteorológicas automáticas em todos os estados brasileiro.

Os arquivos compactados foram baixados do site e salvos localmente em:  
* '~/git/BD/Inmet/'




> **Nota em relação ao Github:** Os arquivos salvos **nesta fase** são maiores do que o tamanho aceito em minha conta, e por isso eles serão ignorados e não disponibilizados.

Filtra arquivos do INMET relacionados as localidades baianas acima listadas e salva localmente

In [16]:
# Carrega biblioteca adcional
import shutil
import zipfile

# Pasta com os arquivos zipados
pasta_zip = '/home/andre301267/git/BD/Inmet/'

# Pasta de destino para os arquivos CSV filtrados no ambiente Linux
pasta_destino_linux = os.path.expanduser('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas/')

# Alternativas a serem verificadas no nome do arquivo
alternativas = ['FORMOSA', 'CORRENTINA', 'EDUARDO', 'BARREIRAS']

# Certifique-se de que a pasta de destino exista ou crie-a
if not os.path.exists(pasta_destino_linux):
    os.makedirs(pasta_destino_linux)

# Itera sobre os arquivos na pasta_zip
for nome_arquivo in os.listdir(pasta_zip):
    caminho_arquivo_zip = os.path.join(pasta_zip, nome_arquivo)

    # Verifica se é um arquivo zip
    if nome_arquivo.lower().endswith('.zip'):
        # Extrai o conteúdo diretamente para a pasta de destino
        with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
            for arquivo in zip_ref.namelist():
                if any(alternativa in arquivo for alternativa in alternativas) and "_BA_" in arquivo and arquivo.lower().endswith('.csv'):
                    # Constrói o caminho de destino de forma mais simplificada, sem o sufixo "copiado_"
                    nome_destino = f"{os.path.splitext(os.path.basename(arquivo))[0]}.CSV"
                    caminho_destino = os.path.join(pasta_destino_linux, nome_destino)

                    with open(caminho_destino, 'wb') as destino, zip_ref.open(arquivo) as origem:
                        shutil.copyfileobj(origem, destino)

print("\n\nTransferência concluída. Arquivos filtrados foram salvos em:\n\
\t", pasta_destino_linux,"\n\n")




Transferência concluída. Arquivos filtrados foram salvos em:
	 /home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas/ 




In [17]:
# Diretório dos arquivos
diretorio = '/home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas/'

# Lista para armazenar os DataFrames
df_parte1 = pd.DataFrame()
df_parte2 = pd.DataFrame()

# Iterar sobre os arquivos no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.CSV'):  # Certifique-se de que estamos lidando apenas com arquivos de texto
        caminho_arquivo = os.path.join(diretorio, arquivo)

        # Ler as primeiras 8 linhas
        parte1 = pd.read_csv(caminho_arquivo, nrows=7,sep=';',encoding='Latin-1')

        # Ler a partir da linha 9
        parte2 = pd.read_csv(caminho_arquivo, skiprows=8,sep=';',encoding='Latin-1').assign(Estação=parte1.iloc[1,1])

        # Adicionar DataFrames às listas
        df_parte1=pd.concat([df_parte1,parte1],axis=1)
        df_parte2=pd.concat([df_parte2,parte2])

# Formata arquivo contendo ID das estações
df_parte1=df_parte1.transpose().drop(columns=[5,6]).drop_duplicates(subset=[1]).drop_duplicates(subset=[2])\
.reset_index(drop=True)

df_parte1.columns=df_parte1.iloc[0]

df_parte1=df_parte1.iloc[1:]

df_parte1.to_csv('/home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/estações.csv',index=False)
df_parte2.to_csv('/home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas1.csv',index=False)

In [18]:
print('\n\nApós extraídos, se verificou que as tabelas continham formato misto, onde a identificação da estação\n\
dividia espaço com os dados meteorológicos coletados.\n\n\
Por fim a  tabela de chuvas ficou assim:\n\n')



Após extraídos, se verificou que as tabelas continham formato misto, onde a identificação da estação
dividia espaço com os dados meteorológicos coletados.

Por fim a  tabela de chuvas ficou assim:




In [19]:
display(df_parte2)

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,Estação,Data,Hora UTC,RADIACAO GLOBAL (Kj/m²)
0,2002-01-01,00:00,0,"960,2","960,2","959,5",-9999,"22,6","20,5","22,9",...,86.0,88.0,68,"2,8","1,5",NaN,BARREIRAS,NaN,NaN,NaN
1,2002-01-01,01:00,0,"960,1","960,3","960,1",-9999,"22,4","20,7","22,7",...,88.0,90.0,44,"3,2","1,9",NaN,BARREIRAS,NaN,NaN,NaN
2,2002-01-01,02:00,0,"960,1","960,2",960,-9999,"22,3","20,7","22,6",...,87.0,90.0,63,"4,8","2,7",NaN,BARREIRAS,NaN,NaN,NaN
3,2002-01-01,03:00,0,"959,4","960,1","959,4",-9999,"22,4","20,4","22,4",...,89.0,89.0,41,"5,2",2,NaN,BARREIRAS,NaN,NaN,NaN
4,2002-01-01,04:00,0,"959,2","959,4","959,2",-9999,22,"20,7","22,4",...,88.0,92.0,3,"2,6",",3",NaN,BARREIRAS,NaN,NaN,NaN


Remove o diretório com os arquivos extraídos e ainda separados sobre chuvas

In [21]:
import shutil

diretorio = '/home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas/'

try:
    shutil.rmtree(diretorio)
    #print(f"O diretório '{diretorio}' e todo o seu conteúdo foram removidos com sucesso.")
except OSError as e:
    print(f"Falha ao remover o diretório '{diretorio}': {e}")
